### Modelado 

1. Se visualizarán las matrices ya normalizadas: 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_data_matrix(file_path):
    """
    Carga una matriz de datos desde un archivo CSV y extrae la etiqueta del nombre completo del archivo.

    Parameters:
    - file_path (str): La ruta al archivo CSV.

    Returns:
    - tuple:
        - np.ndarray: Matriz de datos cargada.
        - str: La etiqueta extraída del nombre completo del archivo.
    """
    # Cargar datos desde el archivo CSV
    df = pd.read_csv(file_path)

    # Extraer la etiqueta del nombre completo del archivo, sin la extensión .csv
    label = os.path.basename(file_path).split('_')[0]

    return df.values, label


In [3]:
file_path = r'C:\Users\franc\Downloads\saved_matrix\saved_matrix_normalize\CA_15o14006.csv'
df, label = load_data_matrix(file_path)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import math

def plot_data_matrix(data_matrix, title="Data Matrix"):
    """
    Grafica una matriz de datos donde cada columna es un segmento de la función,
    mostrando hasta 5 segmentos por gráfica y permitiendo interactividad con widgets.

    Parameters:
    - data_matrix (np.ndarray): La matriz de datos a graficar.
    - title (str): Título del gráfico.

    Outputs:
    - None
    """
    num_segments = data_matrix.shape[1]
    num_plots = math.ceil(num_segments / 5)  # Calcula el número necesario de gráficos

    def plot_subset(page=0):
        plt.figure(figsize=(15, 8))  # Aumentar el tamaño de la gráfica
        start = page * 5
        end = min(start + 5, num_segments)
        for i in range(start, end):
            plt.plot(data_matrix[:, i], label=f'Segment {i+1}')
        plt.title(f"{title} - Page {page+1}")
        plt.xlabel('Time Points')
        plt.ylabel('Amplitude')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Crear un slider interactivo para navegar entre las diferentes páginas de gráficos
    interact(plot_subset, page=IntSlider(min=0, max=num_plots-1, step=1, value=0, description="Page"))

# Configurando el entorno de matplotlib para interactividad
%matplotlib widget

In [23]:
# !pip install jupyterlab ipympl


  Attempting uninstall: pywinpty
    Found existing installation: pywinpty 2.0.2
    Uninstalling pywinpty-2.0.2:
      Successfully uninstalled pywinpty-2.0.2


In [24]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib

An error occurred.
ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
See the log file for details:  C:\Users\franc\AppData\Local\Temp\jupyterlab-debug-ie2p5owy.log


In [5]:
plot_data_matrix(df, title="Data Matrix")

interactive(children=(IntSlider(value=0, description='Page', max=11), Output()), _dom_classes=('widget-interac…

In [6]:
from sklearn.model_selection import train_test_split

def prepare_train_test_data(directory_path, test_size=0.2):
    """
    Prepara los datos de entrenamiento y prueba a partir de los archivos en un directorio.

    Parameters:
    - directory_path (str): Ruta del directorio que contiene los archivos CSV.
    - test_size (float): Proporción del conjunto de prueba.

    Returns:
    - tuple: Cuatro elementos (X_train, X_test, y_train, y_test).
    """
    X, y = [], []
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            label = filename.split('_')[0]  # Asumiendo que la etiqueta está al inicio del nombre del archivo
            file_path = os.path.join(directory_path, filename)
            data_matrix = load_data_matrix(file_path)
            X.append(data_matrix)
            y.append(label)
    
    X = np.array(X)
    y = np.array(y)

    return train_test_split(X, y, test_size=test_size, random_state=42)


In [7]:
directory_path = r'C:\Users\franc\Downloads\saved_matrix\saved_matrix_normalize'

In [10]:
X_train, X_test, y_train, y_test = prepare_train_test_data(directory_path, test_size=0.2)

C:\Users\franc\AppData\Local\Temp\ipykernel_5128\1163801982.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [17]:
# Asumiendo que X_train y X_test necesitan ser transformados
X_train_lstm = np.expand_dims(X_train, axis=-1)  # Agrega una nueva dimensión al final para las características
X_test_lstm = np.expand_dims(X_test, axis=-1)    # Igual para X_test

# Ahora, X_train y X_test deberían tener la forma (num_samples, num_timesteps, 1)

In [18]:
input_shape = X_train_lstm.shape[1:]

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, TimeDistributed
from tensorflow.keras.metrics import Recall, Precision, F1Score

def build_lstm_model(input_shape):
    """
    Construye y retorna un modelo LSTM.

    Parameters:
    - input_shape (tuple): Forma de los datos de entrada (timesteps, features).

    Returns:
    - tensorflow.keras.Model: El modelo LSTM compilado.
    """
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision(), F1Score()])
    return model

In [19]:
# Construir el modelo LSTM
model_lstm = build_lstm_model(input_shape)

# Entrenar el modelo
model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [12]:


def build_gru_model(input_shape):
    """
    Construye y retorna un modelo GRU.

    Parameters:
    - input_shape (tuple): Forma de los datos de entrada (timesteps, features).

    Returns:
    - tensorflow.keras.Model: El modelo GRU compilado.
    """
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=input_shape),
        GRU(50),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision(), F1Score()])
    return model

def build_rnn_model(input_shape):
    """
    Construye y retorna un modelo RNN simple.

    Parameters:
    - input_shape (tuple): Forma de los datos de entrada (timesteps, features).

    Returns:
    - tensorflow.keras.Model: El modelo RNN compilado.
    """
    model = Sequential([
        SimpleRNN(50, return_sequences=True, input_shape=input_shape),
        SimpleRNN(50),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision(), F1Score()])
    return model
